In [1]:
import autograd.numpy as np
from autograd import grad

In [2]:
class NANDNeuron():
    def __init__(self, n):
        self.n = n
        self.epsilons = np.full((n), 0.0) #np.random.uniform(-1.0, 1.001, n)#
        self.effector = -1
        
    def __repr__(self):
        return str(self.epsilons)
        
    def __beforePresent__(self):
        self.effector = -1
        self.delta = 0
        self.grad = np.zeros(self.n)
    
    def setCheckGrad(self, g):
        self.checkGrad = g
        
    def setGrad(self, g):
        self.grad += g
        
    def getCheckGrad(self):
        return self.checkGrad
    
    def getGrad(self):
        return self.grad
    
    def getDelta(self):
        return self.delta
    
    def setDelta(self, delta):
        self.delta = delta
    
    def present(self, inputs):
        self.__beforePresent__()
        
        self.inputs = inputs
        
        mus = np.array(list(map(lambda x: self.epsilons[x] + inputs[x] - self.epsilons[x] * inputs[x], range(0, len(inputs)))))
        self.mus = mus
#         mus = np.array(list(map(lambda x: np.power(inputs[x], (1-self.epsilons[x])), range(0, len(inputs)))))
        z = np.product(mus)
        
        y = 1 - z#np.power(np.e, z)
        self.output = y
        
        return self.output

    def getInput(self):
        return self.inputs
    
    def getMus(self):
        return self.mus
    
    def getOutput(self):
        return self.output
    
    def updateWeights(self, grad):
        self.epsilons = self.epsilons - grad
        for i in range(0, len(self.epsilons)):
            if self.epsilons[i] > 1:
                self.epsilons[i] = 1
            elif self.epsilons[i] < 0:
                self.epsilons[i] = 0
    
    def getWeights(self):
        return self.epsilons
    
    def getEffector(self):
        return self.effector

class NANDLayer():
    def __init__(self, inputs, nodes):
        self.layer = []
        
        for i in range(0, nodes):
            self.layer.append(NANDNeuron(inputs))
            
    def __repr__(self):
        s = ""
        for l in self.layer:
            s += (str(l) + " ,")
            
        return s

    def getLayer(self):
        return self.layer
    
    def setFolowingLayer(self, l):
        self.folowingLayer = l
        
    def present(self, inputs):
        out = []
        for n in self.layer:
            out.append(n.present(inputs))
        
        return out
    
    def backprop(self, prediction, target, output=False):
        for n in range(0, len(self.layer)):
            grad = None
            if output:
                grad = gradientOutputLayer(self.layer[n], target, prediction)
            else:
                grad = gradientHiddenLayer(self.layer[n], n, self.folowingLayer)
                
            self.layer[n].setGrad(grad)
            self.layer[n].updateWeights(grad * 0.05)
    
class NANDNetwork():
    def __init__(self, nIns, lParams, nOuts):
        self.layers = []
        
        lParams.append(nOuts)
        inputs = nIns
        for l in lParams:
            self.layers.append(NANDLayer(inputs, l))
            inputs = l
                    
        for i in range(1, len(self.layers)):
            self.layers[i-1].setFolowingLayer(self.layers[i])
        
    def __repr__(self):
        s = ""
        for l in range(0, len(self.layers)):
            s += ("Layer " + str(l+1) + " -> " + str(self.layers[l]) + "\n")
            
        return s
        
    def getLayers(self):
        return self.layers
        
    def fowardprop(self, inputs):
        for l in self.layers:
            inputs = l.present(inputs)
            
        return inputs[0]
            
    def backprop(self, prediction, target):
        for i in range(len(self.layers)-1, -1, -1):
            layer = self.layers[i]
            layer.backprop(prediction, target, i==(len(self.layers) - 1))
    
    
def MSE(network, data, targets, p=False):
    expected = np.array(list(map(lambda x: network.fowardprop(x), data)))
    if p:
        print(expected)
    return (1.0/2.0) * np.sum(np.power(np.subtract(expected, targets), 2))

def gradientHiddenLayer(neuron, neuronNumber, folowingLayer):
    numWeights = len(neuron.getWeights())
    grad = np.zeros(numWeights)

    delta = 0
    for i in range(0, len(folowingLayer.getLayer())):
        fn = folowingLayer.getLayer()[i]
        delta += fn.getDelta() * -(1-fn.getWeights()[neuronNumber]) * np.product(np.delete(fn.getMus(), neuronNumber))
        
    neuron.setDelta(delta)
    
    for i in range(0, numWeights):
        grad[i] = delta * -(1-neuron.getInput()[i]) * np.product(np.delete(neuron.getMus(), i))
    
    return grad
    
    
def gradientOutputLayer(neuron, target, prediction):
    numWeights = len(neuron.getWeights())
    grad = np.zeros(numWeights)

    delta = -(target - prediction)
    for i in range(0, numWeights):
        grad[i] = delta * -(1-neuron.getInput()[i]) * np.product(np.delete(neuron.getMus(), i))
        
    neuron.setDelta(delta)
    
    return grad


In [3]:
def trainNANDNetwork(data, targets, inputNodes, hLayers, outNodes):
    network = NANDNetwork(inputNodes, hLayers, outNodes)
    print(network)
    print("Initial Loss: ", MSE(network, data, targets))
    
    pterb = 0.0000001
    
    for i in range(1, 10000):
        if i%1000 == 0:
            print("Iteration -> " + str(i) + " : " + str(MSE(network, data, targets)))
            
        for i in range(0, len(network.getLayers())):
            layer = network.getLayers()[i]
            for j in range(0, len(layer.getLayer())):
                neuron = layer.getLayer()[j]
                grad = np.zeros(len(neuron.getWeights()))
            
                for k in range(0, len(neuron.getWeights())):
                    g = np.zeros(len(neuron.getWeights()))
                    g[k] = -pterb

                    oldSSE = MSE(network, data, targets)
                    neuron.updateWeights(g)
                    newSSE = MSE(network, data, targets)
                    neuron.updateWeights(-g)
                
                    grad[k] = (newSSE - oldSSE)/pterb
                
                neuron.updateWeights(grad * 0.02)
            
    print("Trained Loss: ", MSE(network, data, targets, True))
    return network
            
            
        
def checkGrad(pterb, threshold, inputNodes, hLayers, outNodes):
    network = NANDNetwork(inputNodes, hLayers, outNodes)
    
    print("Computing Numerical Grads")
    for i in range(0, len(network.getLayers())):
        layer = network.getLayers()[i]
        for j in range(0, len(layer.getLayer())):
            neuron = layer.getLayer()[j]
            grad = np.zeros(len(neuron.getWeights()))
            
            for k in range(0, len(neuron.getWeights())):
                g = np.zeros(len(neuron.getWeights()))
                g[k] = -pterb
                
                oldSSE = MSE(network, data, targets)
                neuron.updateWeights(g)
                newSSE = MSE(network, data, targets)
                neuron.updateWeights(-g)
                
                grad[k] = (newSSE - oldSSE)/pterb
                
#             print(grad)
            neuron.setCheckGrad(grad)
    
    print("Running Back Prop")
    for j in range(0, len(data)):
        prediction = network.fowardprop(data[j])
        network.backprop(prediction, targets[j])
        
        
    print("Checking Grad")
    for i in range(0, len(network.getLayers())):
        layer = network.getLayers()[i]
        for j in range(0, len(layer.getLayer())):
            neuron = layer.getLayer()[j]
            
            diff = np.absolute(neuron.getCheckGrad() - neuron.getGrad())
            for k in diff:
                if k > threshold:
                    print("GRAD WRONG[ " + str(i) + "," + str(j) + " ]: Got " + str(neuron.getGrad()) + " Should be " + str(neuron.getCheckGrad()))
                    break
                
                

# NAND Gate

In [4]:
dataNAND = np.array([[0.0,0.0],[1.0,0.0],[0.0,1.0],[1.0,1.0]])
targetsNAND = np.array([1.0,1.0,1.0,0.0])
trainNANDNetwork(dataNAND, targetsNAND, 2, [], 1)

Layer 1 -> [ 0.  0.] ,

Initial Loss:  0.0
Iteration -> 1000 : 0.0
Iteration -> 2000 : 0.0
Iteration -> 3000 : 0.0
Iteration -> 4000 : 0.0
Iteration -> 5000 : 0.0
Iteration -> 6000 : 0.0
Iteration -> 7000 : 0.0
Iteration -> 8000 : 0.0
Iteration -> 9000 : 0.0
[ 1.  1.  1.  0.]
Trained Loss:  0.0


Layer 1 -> [ 0.  0.] ,

# NOT Gate

In [6]:
dataNOT = np.array([[0.0, 1.0], [1.0, 1.0]])
targetsNOT = np.array([1.0, 0.0])
trainNANDNetwork(dataNOT, targetsNOT, 2, [], 1)

Layer 1 -> [ 0.  0.] ,

Initial Loss:  0.0
Iteration -> 1000 : 0.0
Iteration -> 2000 : 0.0
Iteration -> 3000 : 0.0
Iteration -> 4000 : 0.0
Iteration -> 5000 : 0.0
Iteration -> 6000 : 0.0
Iteration -> 7000 : 0.0
Iteration -> 8000 : 0.0
Iteration -> 9000 : 0.0
[ 1.  0.]
Trained Loss:  0.0


Layer 1 -> [ 0.  0.] ,

# AND Gate

In [8]:
dataAND = np.array([[0.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 1.0]])
targetsAND = np.array([0.0, 0.0, 0.0, 1.0])
trainNANDNetwork(dataAND, targetsAND, 2, [1], 1)

Layer 1 -> [ 0.  0.] ,
Layer 2 -> [ 0.] ,

Initial Loss:  0.0
Iteration -> 1000 : 0.0
Iteration -> 2000 : 0.0
Iteration -> 3000 : 0.0
Iteration -> 4000 : 0.0
Iteration -> 5000 : 0.0
Iteration -> 6000 : 0.0
Iteration -> 7000 : 0.0
Iteration -> 8000 : 0.0
Iteration -> 9000 : 0.0
[ 0.  0.  0.  1.]
Trained Loss:  0.0


Layer 1 -> [ 0.  0.] ,
Layer 2 -> [ 0.] ,

# OR Gate

In [9]:
dataOR = np.array([[0.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 1.0]])
targetsOR = np.array([0.0, 1.0, 1.0, 1.0])
trainNANDNetwork(dataOR, targetsOR, 2, [2], 1)

Layer 1 -> [ 0.  0.] ,[ 0.  0.] ,
Layer 2 -> [ 0.  0.] ,

Initial Loss:  1.0
Iteration -> 1000 : 0.156097848728
Iteration -> 2000 : 0.144529447194
Iteration -> 3000 : 9.80486407132e-15
Iteration -> 4000 : 9.80486407132e-15
Iteration -> 5000 : 9.80486407132e-15
Iteration -> 6000 : 9.80486407132e-15
Iteration -> 7000 : 9.80486407132e-15
Iteration -> 8000 : 9.80486407132e-15
Iteration -> 9000 : 9.80486407132e-15
[ 0.         0.9999999  0.9999999  1.       ]
Trained Loss:  9.80486407132e-15


Layer 1 -> [ 0.9999999  0.       ] ,[ 0.         0.9999999] ,
Layer 2 -> [ 0.  0.] ,

# A Implys B Gate

In [10]:
dataI = np.array([[0.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 1.0]])
targetsI = np.array([1.0, 1.0, 0.0, 1.0])
trainNANDNetwork(dataI, targetsI, 2, [2], 1)

Layer 1 -> [ 0.  0.] ,[ 0.  0.] ,
Layer 2 -> [ 0.  0.] ,

Initial Loss:  1.0
Iteration -> 1000 : 0.250000005659
Iteration -> 2000 : 0.250000005659
Iteration -> 3000 : 0.250000005659
Iteration -> 4000 : 0.250000005659
Iteration -> 5000 : 0.250000005659
Iteration -> 6000 : 0.250000005659
Iteration -> 7000 : 0.250000005659
Iteration -> 8000 : 0.250000005659
Iteration -> 9000 : 0.250000005659
[ 0.49999995  0.99999996  0.49999996  0.99999996]
Trained Loss:  0.250000005659


Layer 1 -> [ 1.         0.3868138] ,[ 0.9999999   0.18672872] ,
Layer 2 -> [  3.88484800e-06   1.14593043e-02] ,